In [28]:
# Yihao Zhong Larry, yz7654, date: 02/23/2024 - onwards
# 

## Problem 1 -  Softmax Activation Function

### 1.1. logarithmic derivative


Given that for each real-valued outputs $v_1$, $v_2$, ..., $v_k$, $o_i = \frac{exp(v_i)}{\sum^k_{j=1}{exp(v_j)}}$

For the case where $i=j$, we consider the derivative of $o_i$ with respect to $v_i$.

Take logarithm: $$ln(o_i) = ln(\frac{exp(v_i)}{\sum^k_{j=1}{exp(v_j)}}) = v_i-ln(\sum^k_{j=1}{exp(v_j)})$$

Taking the derivative of both sides with respect to $v_i$: 
$$\frac{d}{dv_i} ln(o_i) = 1 - \frac{1}{\sum^k_{j=1}{exp(v_j)}}e^{v_i}$$

Since $o_i = \frac{exp(v_i)}{\sum^k_{j=1}{exp(v_j)}}$, we have $$ \frac{d}{dv_i} ln(o_i) = 1-o_i $$

To find $\frac{\partial{o_i}}{\partial{v_j}}$, we know that $\frac{\partial}{\partial{x_j}}ln(f(x)) = \frac{1}{f(x)}\frac{\partial{f(x)}}{\partial{x_j}}$ by logarithmic derivative.

Therefore, $$\frac{\partial{o_i}}{\partial{v_j}} = o_i(1-o_i)$$


For the case $i \neq j$, we have start similarly by applying the logarithm to $i$, but take the derivative with respect to $v_j$.

Take logarithm: $$ln(o_i) = ln(\frac{exp(v_i)}{\sum^k_{j=1}{exp(v_j)}})$$

Taking the derivative with respect to $v_j$: $$\frac{d}{dv_j} ln(o_i) =-\frac{e^{v_j}}{\sum^k_{j=1}{exp(v_j)}}$$ 

Simply this by definition of $v_j$ we have: $$\frac{d}{dv_j} ln(o_i) = -o_j$$

To find $\frac{\partial{o_i}}{\partial{v_j}}$, we know that $\frac{\partial}{\partial{x_j}}ln(f(x)) = \frac{1}{f(x)}\frac{\partial{f(x)}}{\partial{x_j}}$ by logarithmic derivative.

Therefore, $$\frac{\partial{o_i}}{\partial{v_j}} = -o_io_j$$

### 1.2. 

## Problem 2 -  Neural Network Training and Backpropagation

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('ex3data1.mat')
X = data['X']
y = data['y']

print(X.shape, y.shape)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
print(y_onehot.shape)

(5000, 400) (5000, 1)


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

### 2.1.

In [ ]:
def scaled_sigmoid(z):
    return 1 / (1 + np.exp(-2 * z))


### 2.2. forward_propagate()

In [30]:
def forward_propagate(X, theta1, theta2, theta3):
    m = X.shape[0]
    
    # Input layer activations (adding bias unit)
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)

    z2 = a1 * (theta1.T)
    a2 = np.insert(scaled_sigmoid(z2), 0, values=np.ones(m), axis=1)
    
    z3 = a2 * (theta2.T)
    a3 = np.insert(scaled_sigmoid(z3), 0, values=np.ones(m), axis=1)
    
    z4 = a3 * (theta3.T)
    h = scaled_sigmoid(z4)
    
    return a1, z2, a2, z3, a3, z4, h


### 2.3. cost()

In [31]:
def cost(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    end_theta1 = hidden_size1 * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size2 * (hidden_size1 + 1)
    theta1 = np.reshape(params[:end_theta1], (hidden_size1, input_size + 1))
    theta2 = np.reshape(params[end_theta1:end_theta2], (hidden_size2, hidden_size1 + 1))
    theta3 = np.reshape(params[end_theta2:], (num_labels, hidden_size2 + 1))
    
    # just need the output of the last layer
    _, _, _, _, _, _, h = forward_propagate(X, theta1, theta2, theta3)
    
    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)
    J = J / m

    return J

def cost_with_regulazation(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    end_theta1 = hidden_size1 * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size2 * (hidden_size1 + 1)
    theta1 = np.reshape(params[:end_theta1], (hidden_size1, input_size + 1))
    theta2 = np.reshape(params[end_theta1:end_theta2], (hidden_size2, hidden_size1 + 1))
    theta3 = np.reshape(params[end_theta2:], (num_labels, hidden_size2 + 1))
    
    # just need the output of the last layer
    _, _, _, _, _, _, h = forward_propagate(X, theta1, theta2, theta3)
    
    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)
    J = J / m

    J += (float(learning_rate) / (2 * m)) \
            * (np.sum(np.square(theta1[:, 1:])) \
                + np.sum(np.square(theta2[:, 1:])) + np.sum(np.square(theta3[:, 1:])))
    return J



In [32]:
# Initial setup
input_size = 400  # Size of input layer
hidden_size1 = 20  # Size of first hidden layer
hidden_size2 = 20  # Size of second hidden layer
num_labels = 10  # Size of output layer
learning_rate = 1

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# Randomly initialize parameters for the full network's parameters, adjusted for 3 layers
params_size = (input_size + 1) * hidden_size1 + (hidden_size1 + 1) * hidden_size2 + (hidden_size2 + 1) * num_labels
params = (np.random.random(size=params_size) - 0.5) * 0.25

# Unravel the parameter array into parameter matrices for each layer
end_theta1 = hidden_size1 * (input_size + 1)
end_theta2 = end_theta1 + hidden_size2 * (hidden_size1 + 1)

theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size1, input_size + 1)))
theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size2, hidden_size1 + 1)))
theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, hidden_size2 + 1)))

print(f"Theta1 shape: {theta1.shape}")  
print(f"Theta2 shape: {theta2.shape}")  
print(f"Theta3 shape: {theta3.shape}")  

Theta1 shape: (20, 401)
Theta2 shape: (20, 21)
Theta3 shape: (10, 21)


In [33]:
print("cost without regularization: ", 
      cost(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate))
print("cost with regularization: ", 
      cost_with_regulazation(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate))
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
print(f"a1: {a1.shape}, z2: {z2.shape}, a2: {a2.shape}, z3: {z3.shape}, a3: {a3.shape}, z4: {z4.shape}, h: {h.shape}")

NameError: name 'y_onehot' is not defined

### 2.4. sigmoid gradient



In [117]:

def scaled_sigmoid_gradient(z):
    return np.multiply(scaled_sigmoid(z), (1 - scaled_sigmoid(z)))

### 2.5. backprop()

In [118]:
def get_theta_shapes(input_size, hidden_size1, hidden_size2, num_labels):
    """
    Returns the shapes of theta matrices for each layer in the network.
    """
    theta1_shape = (hidden_size1, input_size + 1)
    theta2_shape = (hidden_size2, hidden_size1 + 1)
    theta3_shape = (num_labels, hidden_size2 + 1)
    return theta1_shape, theta2_shape, theta3_shape

def reshape_params(params, shapes):
    """
    Reshapes the flattened parameter array into individual theta matrices.
    """
    theta1_shape, theta2_shape, theta3_shape = shapes
    size1 = theta1_shape[0] * theta1_shape[1]
    size2 = size1 + theta2_shape[0] * theta2_shape[1]
    
    theta1 = params[:size1].reshape(theta1_shape)
    theta2 = params[size1:size2].reshape(theta2_shape)
    theta3 = params[size2:].reshape(theta3_shape)
    
    return theta1, theta2, theta3


In [119]:
def backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta_shapes = get_theta_shapes(input_size, hidden_size1, hidden_size2, num_labels)
    theta1, theta2, theta3 = reshape_params(params, theta_shapes)
    
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
    
    # Gradients initialization
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    # Compute cost
    J = cost_with_regulazation(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate)
    
    # Compute gradients (Backpropagation)
    for t in range(m):
        a1t = a1[t,:] 
        z2t = z2[t,:] 
        a2t = a2[t,:]  
        ht = h[t,:]  
        yt = y[t,:]
        
        d4 = ht - yt
        d3 = np.multiply((theta3.T * d4.T).T, scaled_sigmoid_gradient(np.insert(z3[t], 0, values=1)))
        d2 = np.multiply((theta2.T * d3[:,1:].T).T, scaled_sigmoid_gradient(np.insert(z2[t], 0, values=1)))
        
        delta3 += d4.T * a3[t]
        delta2 += d3[:,1:].T * a2t  
        delta1 += d2[:,1:].T * a1t  
    
    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m
    
    # Unroll gradients
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))
    
    
    return J, grad

def backprop_with_reg(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta_shapes = get_theta_shapes(input_size, hidden_size1, hidden_size2, num_labels)
    theta1, theta2, theta3 = reshape_params(params, theta_shapes)
    
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
    
    # Gradients initialization
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    # Compute cost
    J = cost_with_regulazation(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate)
    
    # Compute gradients (Backpropagation)
    for t in range(m):
        a1t = a1[t,:] 
        z2t = z2[t,:] 
        a2t = a2[t,:]  
        ht = h[t,:]  
        yt = y[t,:]
        
        d4 = ht - yt
        d3 = np.multiply((theta3.T * d4.T).T, scaled_sigmoid_gradient(np.insert(z3[t], 0, values=1)))
        d2 = np.multiply((theta2.T * d3[:,1:].T).T, scaled_sigmoid_gradient(np.insert(z2[t], 0, values=1)))
        
        delta3 += d4.T * a3[t]
        delta2 += d3[:,1:].T * a2t  
        delta1 += d2[:,1:].T * a1t  
    
    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m
    
    delta1[:, 1:] += (theta1[:, 1:] * learning_rate) / m
    delta2[:, 1:] += (theta2[:, 1:] * learning_rate) / m
    delta3[:, 1:] += (theta3[:, 1:] * learning_rate) / m

    # Unroll gradients
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))
    
    return J, grad


In [120]:
J, grad = backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
print(f"Backpropagation cost without regularization: {J}, grad shape: {grad.shape}")

J_reg, grad_reg = backprop_with_reg(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
print(f"Backpropagation cost with regularization: {J_reg}, grad shape: {grad_reg.shape}")

Backpropagation cost without regularization: 6.807714127811414, grad shape: (8650,)


Backpropagation cost with regularization: 6.807714127811414, grad shape: (8650,)


### 2.6. & 2.7. Train

In [121]:
from scipy.optimize import minimize

options = {'maxiter': 250}


result = minimize(fun=backprop_with_reg, x0=params, 
                  args=(input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate), 
                  method='TNC', jac=True, options=options)


params_optimized = result.x
cost_optimized = result.fun

print(result)
print(f"Optimized cost: {cost_optimized}")


/var/folders/v1/6k_h9wg90q56lxft6dxycw200000gn/T/ipykernel_26624/488540519.py:6: DeprecationWarning: 'maxiter' has been deprecated in favor of 'maxfun' and will be removed in SciPy 1.11.0.
  result = minimize(fun=backprop_with_reg, x0=params,


 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 0.144595091159654
       x: [ 3.798e-02 -3.399e-04 ... -1.601e+00 -1.268e+00]
     nit: 33
     jac: [-1.204e-04 -6.798e-08 ...  3.294e-04  4.233e-04]
    nfev: 250
Optimized cost: 0.144595091159654


In [124]:
end_theta1 = hidden_size1 * (input_size + 1)
end_theta2 = end_theta1 + hidden_size2 * (hidden_size1 + 1)


theta1 = np.matrix(np.reshape(result.x[:end_theta1], (hidden_size1, input_size + 1)))
theta2 = np.matrix(np.reshape(result.x[end_theta1:end_theta2], (hidden_size2, hidden_size1 + 1)))
theta3 = np.matrix(np.reshape(result.x[end_theta2:], (num_labels, hidden_size2 + 1)))


X = np.matrix(X)
# X = np.insert(X, 0, values=np.ones(X.shape[0]), axis=1)

a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)


y_pred = np.array(np.argmax(h, axis=1) + 1)

# y = np.squeeze(np.array(y))

correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(correct) / float(len(correct)))
print('Accuracy = {0}%'.format(accuracy * 100))

Accuracy = 99.83999999999999%


### 2.8. 

We achieve an accuracy of 99.86% with a 3-layer neural network compared to 99.2% with a 2-layer network from the notebooks. Basically, we add an additional hidden layer increases the complexity of the model. This means the network has more parameters (weights and biases) to adjust during training, allowing it to capture more intricate patterns in the data. A deeper network can learn more complex function. By adding more layers, the network can create more complex representations of the input data.

## Problem 3 - Weight Initialization, Dead Neurons, Leaky ReLU

### 3.1. 


In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras import backend as K
import keras
from matplotlib import pyplot as plt
from matplotlib import rcParamsDefault
import tensorflow as tf
## use of code from github.com/intoli/intoli-article-materials/blob/master/articles/neural-network-initialization/utils.py

def grid_axes_it(n_plots, n_cols=3, enumerate=False, fig=None):
    """
    Iterate through Axes objects on a grid with n_cols columns and as many
    rows as needed to accommodate n_plots many plots.

    Args:
        n_plots: Number of plots to plot onto figure.
        n_cols: Number of columns to divide the figure into.
        fig: Optional figure reference.

    Yields:
        n_plots many Axes objects on a grid.
    """
    n_rows = int(n_plots / n_cols + int(n_plots % n_cols > 0))

    if not fig:
        default_figsize = rcParamsDefault['figure.figsize']
        fig = plt.figure(figsize=(
            default_figsize[0] * n_cols,
            default_figsize[1] * n_rows
        ))

    for i in range(1, n_plots + 1):
        ax = plt.subplot(n_rows, n_cols, i)
        yield ax


def create_mlp_model(
    n_hidden_layers,
    dim_layer,
    input_shape,
    n_classes,
    kernel_initializer,
    bias_initializer,
    activation,
):
    """Create Multi-Layer Perceptron with given parameters."""
    model = Sequential()
    model.add(Dense(dim_layer, input_shape=input_shape, kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    for i in range(n_hidden_layers):
        model.add(Dense(dim_layer, activation=activation, kernel_initializer=kernel_initializer,
                        bias_initializer=bias_initializer))
    model.add(Dense(n_classes, activation='softmax', kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    return model


def create_cnn_model(input_shape, num_classes, kernel_initializer='glorot_uniform',
                     bias_initializer='zeros'):
    """Create CNN model similar to
       https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py."""
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape,
                     kernel_initializer=kernel_initializer,
                     bias_initializer=bias_initializer))
    model.add(Conv2D(64, (3, 3), activation='relu',
                     kernel_initializer=kernel_initializer,
                     bias_initializer=bias_initializer))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu',
                    kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax',
                    kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))
    return model


def compile_model(model):
    model.compile(loss = keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.legacy.RMSprop(),
                  metrics=['accuracy'])
    return model


def get_init_id(init):
    """
    Returns string ID summarizing initialization scheme and its parameters.

    Args:
        init: Instance of some initializer from keras.initializers.
    """
    try:
        init_name = str(init).split('.')[2].split(' ')[0]
    except:
        init_name = str(init).split(' ')[0].replace('.', '_')

    param_list = []
    config = init.get_config()
    for k, v in config.items():
        if k == 'seed':
            continue
        param_list.append('{k}-{v}'.format(k=k, v=v))
    init_params = '__'.join(param_list)

    return '|'.join([init_name, init_params])


def get_activations(model, x, mode=0.0):
    """Extract activations with given model and input vector x."""
    outputs = [layer.output for layer in model.layers]
    activations = K.function([model.input], outputs)
    output_elts = activations([x, mode])
    return output_elts


class LossHistory(keras.callbacks.Callback):
    """A custom keras callback for recording losses during network training."""

    def on_train_begin(self, logs={}):
        self.losses = []
        self.epoch_losses = []
        self.epoch_val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

    def on_epoch_end(self, epoch, logs={}):
        self.epoch_losses.append(logs.get('loss'))
        self.epoch_val_losses.append(logs.get('val_loss'))

In [24]:
def calculate_gradients(network, inputs, targets):

    loss_function = tf.keras.losses.CategoricalCrossentropy()
    

    with tf.GradientTape(persistent= True) as gradient_tape:
        predictions = network(inputs)  
        loss = loss_function(targets, predictions) 
    
    
    gradients_list = []

    for layer in network.layers:
        if layer.trainable:
            for weight in layer.trainable_weights:
                if 'bias' not in weight.name:
                    weight_gradient = gradient_tape.gradient(loss, weight)
                    gradients_list.append(weight_gradient)
    
    return gradients_list

In [25]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from keras import initializers
from keras.datasets import mnist



seed = 10

# Number of points to plot
n_train = 1000
n_test = 100
n_classes = 10

# Network params
n_hidden_layers = 5
dim_layer = 100
batch_size = n_train
epochs = 1

# Load and prepare MNIST dataset.
n_train = 60000
n_test = 10000

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = len(np.unique(y_test))
data_dim = 28 * 28

x_train = x_train.reshape(60000, 784).astype('float32')[:n_train]
x_test = x_test.reshape(10000, 784).astype('float32')[:n_train]
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Run the data through a few MLP models and save the activations from
# each layer into a Pandas DataFrame.
rows = []
sigmas = [0.01, 0.1, 0.8]
activations = ['tanh', 'sigmoid']
for activation in activations:
    for stddev in sigmas:
        init = initializers.RandomNormal(mean=0.0, stddev=stddev, seed=seed)
        activation = activation

        model = create_mlp_model(
            n_hidden_layers,
            dim_layer,
            (data_dim,),
            n_classes,
            init,
            'zeros',
            activation
        )
        compile_model(model)
        model.fit(x_train, y_train)
        output_elts = calculate_gradients(model, x_test, y_test)
        n_layers = len(model.layers)
        i_output_layer = n_layers - 1

        for i, out in enumerate(output_elts[:-1]):
            if i > 0 and i != i_output_layer:
                for out_i in out.numpy().ravel()[::20]:
                    rows.append([i, stddev, out_i])

    df = pd.DataFrame(rows, columns=['Hidden Layer', 'Standard Deviation', 'Output'])

    fig = plt.figure(figsize=(12, 6))
    axes = grid_axes_it(len(sigmas), 1, fig=fig)

    for sig in sigmas:
        ax = next(axes)
        ddf = df[df['Standard Deviation'] == sig]
        sns.violinplot(x='Hidden Layer', y='Output', data=ddf, ax=ax, scale='count', inner=None)

        ax.set_xlabel('')
        ax.set_ylabel('')

        ax.set_title(f'{activation}: Gradients Drawn from $N(\mu = 0, \sigma = {sig})$', fontsize=13)

        if sig == sigmas[1]:
            ax.set_ylabel("Gradient")
        if sig != sigmas[-1]:
            ax.set_xticklabels(())
        else:
            ax.set_xlabel("Hidden Layer")
# optimizer `tf.keras.optimizers.RMSprop` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead
    plt.tight_layout()
    plt.show()

 272/1875 [===>..........................] - ETA: 38s - loss: 9.6266 - accuracy: 0.1319

KeyboardInterrupt: 

### 3.2 






In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from keras import initializers
from keras.datasets import mnist



seed = 10

# Number of points to plot
n_train = 1000
n_test = 100
n_classes = 10

# Network params
n_hidden_layers = 5
dim_layer = 100
batch_size = n_train
epochs = 1

# Load and prepare MNIST dataset.
n_train = 60000
n_test = 10000

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = len(np.unique(y_test))
data_dim = 28 * 28

x_train = x_train.reshape(60000, 784).astype('float32')[:n_train]
x_test = x_test.reshape(10000, 784).astype('float32')[:n_train]
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Run the data through a few MLP models and save the activations from
# each layer into a Pandas DataFrame.
rows = []
sigmas = [0.01, 0.1, 0.8]
activations = ['tanh', 'sigmoid']
for activation in activations:
    for stddev in sigmas:
        init = initializers.GlorotNormal(seed=seed)
        activation = activation

        model = create_mlp_model(
            n_hidden_layers,
            dim_layer,
            (data_dim,),
            n_classes,
            init,
            'zeros',
            activation
        )
        compile_model(model)
        model.fit(x_train, y_train)
        output_elts = calculate_gradients(model, x_test, y_test)
        n_layers = len(model.layers)
        i_output_layer = n_layers - 1

        for i, out in enumerate(output_elts[:-1]):
            if i > 0 and i != i_output_layer:
                for out_i in out.numpy().ravel()[::20]:
                    rows.append([i, stddev, out_i])

    df = pd.DataFrame(rows, columns=['Hidden Layer', 'Standard Deviation', 'Output'])

    fig = plt.figure(figsize=(12, 6))
    axes = grid_axes_it(len(sigmas), 1, fig=fig)

    for sig in sigmas:
        ax = next(axes)
        ddf = df[df['Standard Deviation'] == sig]
        sns.violinplot(x='Hidden Layer', y='Output', data=ddf, ax=ax, scale='count', inner=None)

        ax.set_xlabel('')
        ax.set_ylabel('')

        ax.set_title(f'Xavier Initialization, {activation}: Gradients Drawn from $N(\mu = 0, \sigma = {sig})$', fontsize=13)

        if sig == sigmas[1]:
            ax.set_ylabel("Gradient")
        if sig != sigmas[-1]:
            ax.set_xticklabels(())
        else:
            ax.set_xlabel("Hidden Layer")
# optimizer `tf.keras.optimizers.RMSprop` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead
    plt.tight_layout()
    plt.show()

### 3.3. 



### 3.4. Discussion



## Problem 4 - 

### 4.1. Plot

#### 4.1. Written Answer



### 4.2. 



### 4.3. 

### 4.4. 

